In [1]:
!pip install --upgrade datasets

# !pip install -q tensorflow numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [8]:
import tensorflow as tf
import numpy as np
import re
import os
from datasets import load_dataset
from huggingface_hub import notebook_login
from google.colab import drive
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from datasets import load_from_disk

In [ ]:
notebook_login()

In [9]:
# Mount Gdrive to save the dataset
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
BATCH_SIZE = 32
MAX_VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 50
MAX_DATA_SAMPLES = 10000
EMBEDDING_DIM = 256
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2
LSTM_UNITS = 268 #768
MAX_DECODE_LENGTH = SEQUENCE_LENGTH * 2


In [ ]:
#Load dataset from https://huggingface.co/datasets/issai/kazparc
try:
    dataset = load_dataset("issai/kazparc", "kazparc",  trust_remote_code=True)
    print("Success!")
except Exception as e:
    print(e)


In [11]:
# Path where to save
save_path = "/content/drive/MyDrive/kazparc_full_data"

In [ ]:
# Save the dataset
dataset.save_to_disk(save_path)

Saving the dataset (0/2 shards):   0%|          | 0/1742956 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/435742 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/28500 [00:00<?, ? examples/s]

In [ ]:
if os.path.exists(save_path):
    print(f"Path exists: {save_path}")
    print(f"Contents: {os.listdir(save_path)}")
else:
    print(f"Path does NOT exist: {save_path}")

Path exists: /content/drive/MyDrive/kazparc_full_data
Contents: ['dataset_dict.json', 'train', 'validation', 'test']


In [12]:
# Load the saved dataset
dataset = load_from_disk(save_path)

In [14]:
train_data = dataset["train"]

test_data = dataset["test"]
train_data.shape

(1742956, 5)

In [15]:
train_df = train_data.to_pandas()
test_df = test_data.to_pandas()
train_df_kk_en = train_df[train_df.pair == 'kk_en']
test_df_kk_en = test_df[test_df.pair == 'kk_en']
kazakh_sentences = train_df_kk_en['source_lang'][:MAX_DATA_SAMPLES]
english_sentences = train_df_kk_en['target_lang'][:MAX_DATA_SAMPLES]

test_kazakh_sentences = test_df_kk_en['source_lang'][:MAX_DATA_SAMPLES]
test_english_sentences = test_df_kk_en['target_lang'][:MAX_DATA_SAMPLES]

kazakh_sentences.shape

(10000,)

In [16]:
kazakh_sentences.head()

,source_lang
0,«Грамматикалық» деген сөз бұл зерттеуде «осы д...
1,Шығын жүз миллиард доллардан асады деп болжап ...
2,қызыл екі қабатты автобус бұрыштан өтіп бара ж...
3,Жүргізушілердің жұмысы мен тынығу режимін жазу...
4,Егер әкімшілік рәсім жолданымның негізінде қоз...


In [17]:
def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence

processed_kazakh = [preprocess_sentence(s) for s in kazakh_sentences]
processed_english = [preprocess_sentence(s) for s in english_sentences]

test_processed_kazakh = [preprocess_sentence(s) for s in test_kazakh_sentences]
test_processed_english = [preprocess_sentence(s) for s in test_english_sentences]

# print(processed_kazakh[0])
# print(processed_english[0])

In [18]:
#to convert words into integers
def build_tokenizer(sentences):
    tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token='<unk>')
    tokenizer.fit_on_texts(sentences)
    return tokenizer

kazakh_tokenizer = build_tokenizer(processed_kazakh)
english_tokenizer = build_tokenizer(processed_english)

#to make sentences the same length
def texts_to_padded_sequences(tokenizer, texts, maxlen=SEQUENCE_LENGTH):
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
    return padded

kazakh_sequences = texts_to_padded_sequences(kazakh_tokenizer, processed_kazakh)
english_sequences = texts_to_padded_sequences(english_tokenizer, processed_english)

test_kazakh_sequences = texts_to_padded_sequences(kazakh_tokenizer, test_processed_kazakh)
test_english_sequences = texts_to_padded_sequences(english_tokenizer, test_processed_english)

In [19]:
print(kazakh_sequences[0])
print(english_sequences[0])

[   1   80  310    8    1 5250    1 7055 1230 7056   30  430    1 1105
 2071    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[2163    9   89  490  491   79 2870  224    7 1997  444    6    2 7502
    3    2 7503    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [20]:
#SLice to have 1 kaz - 1 eng pair
dataset = tf.data.Dataset.from_tensor_slices((kazakh_sequences, english_sequences))
#to shuffle order of examples during each epoch, discarding smaller last batch
dataset = dataset.shuffle(len(kazakh_sequences)).batch(BATCH_SIZE, drop_remainder=True)

def format_dataset(k_seq, e_seq):
    encoder_input = k_seq
    #teacher forcing method (shifting)
    #last token gets removed (padding)
    decoder_input = e_seq[:, :-1]
    #first token gets removed
    decoder_target = e_seq[:, 1:]
    return (encoder_input, decoder_input), decoder_target

train_dataset = dataset.map(format_dataset).prefetch(tf.data.AUTOTUNE)

In [21]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_kazakh_sequences, test_english_sequences))
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True) # No shuffle for test set
test_dataset = test_dataset.map(format_dataset).prefetch(tf.data.AUTOTUNE)

In [22]:
# ~~~~~ Encoder ~~~~~~
encoder_inputs = tf.keras.Input(shape=(SEQUENCE_LENGTH,), dtype='int32', name='encoder_inputs')
embedding_layer_enc = tf.keras.layers.Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, mask_zero=True, name='encoder_embedding')
x = embedding_layer_enc(encoder_inputs)

encoder_lstm_layer = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(LSTM_UNITS, return_state=True, name='encoder_lstm_main')
    )

output_x, forward_h, forward_c, backward_h, backward_c = encoder_lstm_layer(x)
state_h = tf.keras.layers.Concatenate()([forward_h, backward_h])
state_c = tf.keras.layers.Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]

In [23]:
# ~~~~ Decoder ~~~~~~
decoder_inputs = tf.keras.Input(shape=(SEQUENCE_LENGTH - 1,), dtype='int32', name='decoder_inputs')
embedding_layer_dec = tf.keras.layers.Embedding(input_dim=MAX_VOCAB_SIZE, output_dim=EMBEDDING_DIM, mask_zero=True, name='decoder_embedding')
decoder_x = embedding_layer_dec(decoder_inputs)


decoder_lstm_units = LSTM_UNITS * 2

decoder_lstm = tf.keras.layers.LSTM(decoder_lstm_units, return_sequences=True, return_state=True, name='decoder_lstm_main')
decoder_x, state_h_dec, state_c_dec = decoder_lstm(decoder_x, initial_state=encoder_states)

decoder_dense = tf.keras.layers.Dense(MAX_VOCAB_SIZE, activation='softmax', name='decoder_output_dense')
decoder_outputs = decoder_dense(decoder_x)

In [24]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, 50, 256)   │  2,560,000 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 50)        │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, 49)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 536),     │  1,125,600 │ encoder_embeddin… │
│ (Bidirectional)     │ (None, 268),      │            │ not_equal[0][0]   │
│                     │ (None, 268),      │            │                   │
│                     │ (None, 268),      │            │                   │
│                     │ (None, 268)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 49, 256)   │  2,560,000 │ decoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 536)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 536)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_main   │ [(None, 49, 536), │  1,700,192 │ decoder_embeddin… │
│ (LSTM)              │ (None, 536),      │            │ concatenate[0][0… │
│                     │ (None, 536)]      │            │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_output_den… │ (None, 49, 10000) │  5,370,000 │ decoder_lstm_mai… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,315,792 (50.80 MB)

 Trainable params: 13,315,792 (50.80 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(train_dataset, epochs=5)

Epoch 1/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 897s 3s/step - accuracy: 0.5554 - loss: 6.9414
Epoch 2/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 875s 3s/step - accuracy: 0.7236 - loss: 5.6981
Epoch 3/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 923s 3s/step - accuracy: 0.7341 - loss: 5.2532
Epoch 4/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 920s 3s/step - accuracy: 0.7408 - loss: 4.9281
Epoch 5/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 906s 3s/step - accuracy: 0.7526 - loss: 4.5997


In [27]:
loss, accuracy = model.evaluate(test_dataset)

148/148 ━━━━━━━━━━━━━━━━━━━━ 129s 858ms/step - accuracy: 0.7511 - loss: 5.2438


In [28]:
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)
encoder_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_embedding   │ (None, 50, 256)   │  2,560,000 │ encoder_inputs[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 50)        │          0 │ encoder_inputs[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ [(None, 536),     │  1,125,600 │ encoder_embeddin… │
│ (Bidirectional)     │ (None, 268),      │            │ not_equal[0][0]   │
│                     │ (None, 268),      │            │                   │
│                     │ (None, 268),      │            │                   │
│                     │ (None, 268)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 536)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 536)       │          0 │ bidirectional[0]… │
│ (Concatenate)       │                   │            │ bidirectional[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,685,600 (14.06 MB)

 Trainable params: 3,685,600 (14.06 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
decoder_state_input_h = tf.keras.Input(shape=(decoder_lstm_units,), name='decoder_state_input_h')
decoder_state_input_c = tf.keras.Input(shape=(decoder_lstm_units,), name='decoder_state_input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_single_token_input = tf.keras.Input(shape=(1,), dtype='int32', name='decoder_single_token_input')
decoder_single_token_embedded = embedding_layer_dec(decoder_single_token_input)

decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
    decoder_single_token_embedded, initial_state=decoder_states_inputs
)

decoder_states_outputs = [state_h_inf, state_c_inf]

decoder_outputs_inf_dense = decoder_dense(decoder_outputs_inf)

decoder_inference_model = tf.keras.Model(
    inputs=[decoder_single_token_input] + decoder_states_inputs,
    outputs=[decoder_outputs_inf_dense] + decoder_states_outputs )


decoder_inference_model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_single_tok… │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_embedding   │ (None, 1, 256)    │  2,560,000 │ decoder_single_t… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_inpu… │ (None, 536)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_state_inpu… │ (None, 536)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_lstm_main   │ [(None, 1, 536),  │  1,700,192 │ decoder_embeddin… │
│ (LSTM)              │ (None, 536),      │            │ decoder_state_in… │
│                     │ (None, 536)]      │            │ decoder_state_in… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_output_den… │ (None, 1, 10000)  │  5,370,000 │ decoder_lstm_mai… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,630,192 (36.74 MB)

 Trainable params: 9,630,192 (36.74 MB)

 Non-trainable params: 0 (0.00 B)

In [30]:
MAX_DECODE_LENGTH = SEQUENCE_LENGTH - 1
def translate_sentence(input_sentence):
    preprocessed_sentence = preprocess_sentence(input_sentence)

    input_sequence = texts_to_padded_sequences(kazakh_tokenizer, [preprocessed_sentence], maxlen=SEQUENCE_LENGTH)

    encoder_output_states = encoder_model.predict(input_sequence, verbose=0)

    decoder_current_states = encoder_output_states

    output_tokens = []

    current_token_input = np.array([[0]]) #(batch_size=1, 1)

    # Loop to generate output tokens word by word
    for _ in range(MAX_DECODE_LENGTH):
        predictions_and_states = decoder_inference_model.predict(
            [current_token_input] + decoder_current_states
        )

        predicted_probs = predictions_and_states[0] # (1, 1, vocab_size)
        decoder_current_states = predictions_and_states[1:]

        predicted_id = np.argmax(predicted_probs[0, 0, :])
        output_tokens.append(predicted_id)

        current_token_input = np.array([[predicted_id]])

    reverse_english_word_index = dict([(v, k) for k, v in english_tokenizer.word_index.items()])

    translated_sentence_words = []
    for token_id in output_tokens:
        if token_id != 0:
            translated_sentence_words.append(reverse_english_word_index.get(token_id, '<unk>'))

    return ' '.join(translated_sentence_words)

In [ ]:
while True:
    user_input = input("Enter a Kazakh sentence: ")
    if user_input.lower() == 'e':
        break

    translated = translate_sentence(user_input)
    print(f"Translated English: {translated}")
